# Advanced Data Wrangling I: MultiIndex and Merge in Pandas

Files needed = ('mlb.xlsx', 'CPS_March_2016.csv', 'cities.csv', 'month_legal.csv', 'delinquency.csv', 'state_unemp.csv')

We will often find ourselves with data in which the unit of observation is complex. Pandas helps us deal with this by allowing for many index variables.

Some examples that could use a multiIndex

1. State and county
2. Team and player
3. Industry and firm
4. Country (or person, firm, ...) and time

That last one is important, and one that shows up a lot in economics. We call this *panel data*. Panel data is sometimes called *longitudinal data*. It follows the same firm/person/country over time. We are often interested in how *trends* in economic outcomes are impacted by *changes* in the economic environment, e.g., policies, regulations, technologies, contracts, etc.

MultiIndexes are important. Today, we'll explore how MultiIndexes can be used to retrieve subsets of data and merge datasets.

In [6]:
import pandas as pd                 # load pandas and shorten it to pd
import datetime as dt   

import os
os.getcwd()
os.chdir('/Users/Jackson/Documents/ECON570')
os.getcwd()            # load datetime and shorten it to dt

'/Users/jackson/Documents/ECON570'

Let's bring in some data on National League teams in the MLB. The data include the team name, city, and win proportion for the years 1998-2007. Let's read in the data as is.

In [7]:
mlb = pd.read_excel('data/mlb.xlsx')
mlb.head()

,Unnamed: 0,Phoenix,Atlanta,Chicago,Cincinnati,Denver,Houston,Los Angeles,Miami,Milwaukee,New York,Philadelphia,Pittsburgh,San Diego,San Francisco,St. Louis,Washington
0,NaN,Diamondbacks,Braves,Cubs,Reds,Rockies,Astros,Dodgers,Marilns,Brewers,Mets,Phillies,Pirates,Padres,Giants,Cardinals,Nationals
1,1998.0,0.398773,0.650307,0.552147,0.472393,0.472393,0.625767,0.509202,0.331288,0.453988,0.539877,0.460123,0.423313,0.601227,0.546012,0.509202,0.398773
2,1999.0,0.613497,0.631902,0.411043,0.588957,0.441718,0.595092,0.472393,0.392638,0.453988,0.595092,0.472393,0.478528,0.453988,0.527607,0.460123,0.417178
3,2000.0,0.524691,0.58642,0.401235,0.524691,0.506173,0.444444,0.530864,0.487654,0.450617,0.580247,0.401235,0.425926,0.469136,0.598765,0.58642,0.41358
4,2001.0,0.567901,0.54321,0.54321,0.407407,0.450617,0.574074,0.530864,0.469136,0.419753,0.506173,0.530864,0.382716,0.487654,0.555556,0.574074,0.419753


Yuck. Looks like an excel spreadsheet that is not particularly well-formatted.

The following code cleans the data a bit. I'm just going to wave my magic wand to clean it up with the following block of code, but once we've gotten through the next three lectures, I recommend each of you come back and try to do it yourself from scratch. Look at the raw data, look at how we want the data formatted, and see if you can get from "raw" to "formatted" without any help.

In [9]:
# Read in data with row/column indexing
mlb = pd.read_excel('data/mlb.xlsx', header=[0,1], index_col=0)
# Stack the data appropriately
mlb = mlb.stack(level=[-1,0], future_stack=True).to_frame()
# Rename the column, name the index variables
mlb.rename(columns={0:'win_prop'}, inplace=True)
mlb.index.rename(['year','team','city'] , inplace=True)
# Reset index
mlb.reset_index(['year','team','city'], inplace=True)
mlb.head()

,year,team,city,win_prop
0,1998,Diamondbacks,Phoenix,0.398773
1,1998,Braves,Atlanta,0.650307
2,1998,Cubs,Chicago,0.552147
3,1998,Reds,Cincinnati,0.472393
4,1998,Rockies,Denver,0.472393


Phew! Much better.

## Single Indexing

Before we introduce the concept of MultiIndex, let's consider a single index by `team` only.

In [10]:
mlb_teamindex = mlb.set_index(['team'])
mlb_teamindex.head()

,year,city,win_prop
team,,,
Diamondbacks,1998,Phoenix,0.398773
Braves,1998,Atlanta,0.650307
Cubs,1998,Chicago,0.552147
Reds,1998,Cincinnati,0.472393
Rockies,1998,Denver,0.472393


This is useful, since we can now take subsets of the data by team, and do various calculations.

For example, we can use `.loc[]` to subset the data for the Giants.

In [11]:
mlb_teamindex.loc['Giants']  # ask for the rows corresponding to the Giants

,year,city,win_prop
team,,,
Giants,1998,San Francisco,0.546012
Giants,1999,San Francisco,0.527607
Giants,2000,San Francisco,0.598765
Giants,2001,San Francisco,0.555556
Giants,2002,San Francisco,0.586420
Giants,2003,San Francisco,0.617284
Giants,2004,San Francisco,0.561728
Giants,2005,San Francisco,0.462963
Giants,2006,San Francisco,0.469136


We can also calculate, for example, the average win proportion for the Giants over this time period. (What does `{:3.2f}` do?)

In [12]:
'Giants\' average win proportion from 1998 to 2007: {:3.2f}'.format(mlb_teamindex.loc['Giants','win_prop'].mean())

"Giants' average win proportion from 1998 to 2007: 0.54"

## Multiple Indexing

Let's set up the DataFrame to take both `team` and `year` as the indexes.

In [13]:
mlb.set_index(['team','year'], inplace=True)
mlb.head()

,,city,win_prop
team,year,,
Diamondbacks,1998,Phoenix,0.398773
Braves,1998,Atlanta,0.650307
Cubs,1998,Chicago,0.552147
Reds,1998,Cincinnati,0.472393
Rockies,1998,Denver,0.472393


Take a quick look at the output. The highest level of the index is 'team' (we passed it 'team' first in the list) and the second level is 'year'.

It is more useful to see the data sorted by index order.

In [14]:
mlb.sort_index(inplace=True)
mlb.head()

city  win_prop
team   year                   
Astros 1998  Houston  0.625767
       1999  Houston  0.595092
       2000  Houston  0.444444
       2001  Houston  0.574074
       2002  Houston  0.518519

It is immediately clear that the *order of indexing matters*! Note that `.sort_index()` sorts the data by the highest level of index ('outer index') first, and then the second highest level, and so on.

We can still use `.loc[]` to subset our data. For example, if we want to subset on only the team:

In [15]:
mlb.loc['Rockies']

,city,win_prop
year,,
1998,Denver,0.472393
1999,Denver,0.441718
2000,Denver,0.506173
2001,Denver,0.450617
2002,Denver,0.450617
2003,Denver,0.456790
2004,Denver,0.419753
2005,Denver,0.413580
2006,Denver,0.469136


With a MultiIndex, we can also input a tuple of (team, year) values.

In [16]:
mlb.loc[('Rockies', 2006)]

city          Denver
win_prop    0.469136
Name: (Rockies, 2006), dtype: object

What happens when we try to use `.loc[]` to subset by `year`?

In [17]:
mlb.loc[2006, :]

KeyError: 2006

Unfortunate... it turns out that to do this, we need a new function called `.xs()`

In [ ]:
# Get all of the 2006 observations.
mlb.xs(2006, level = 'year', drop_level=False)

We just took a slice from our data for just one year. This is called a *cross section*.

With `.xs()`, we can partially index on the 'outer index' as well. Suppose we want all the Rockies observations using `.xs()`.

In [18]:
# Get all of the Rockies observations.
mlb.xs('Rockies', level = 'team', drop_level=False)

city  win_prop
team    year                  
Rockies 1998  Denver  0.472393
        1999  Denver  0.441718
        2000  Denver  0.506173
        2001  Denver  0.450617
        2002  Denver  0.450617
        2003  Denver  0.456790
        2004  Denver  0.419753
        2005  Denver  0.413580
        2006  Denver  0.469136
        2007  Denver  0.552147

## Practice: MultiIndex

Let's take data from the [Current Population Survey](https://www.census.gov/programs-surveys/cps.html), which surveys about 60,000 households each month. We will compute some average wages. This is the survey from the United States Bureau of Labor Statistics used to produce official measures of the unemployment rate and many more labor-market indicators. 

We will need to clean up a bit, then work with a multiIndex.

The unit of observation is a person. The variables are:

* `hrwage`: hourly wage
* `educ`: education level
* `female`: 1 if female, 0 if not
* `fulltimely`: 1 if worked full time, 0 if not

Use the line below to load the march cps data, 'CPS_March_2016.csv'.  Note: the missing values are '.'

```python
cps = pd.read_csv('CPS_March_2016.csv', na_values = '.')
```

1. Keep only those with `fulltimely == 1`
2. Keep only those with `5 <= hrwage <= 200`

3. Set the index to 'female' and 'educ', in that order.
4. Sort the index.

5. Report the average wage for `HS diploma/GED` and for `College degree`, regardless of gender.

In [25]:
cps = pd.read_csv('data/CPS_March_2016.csv', na_values = '.')
cps

,hrwage,educ,female,fulltimely
0,20.961538,Some college,0,1.0
1,20.192308,HS diploma/GED,1,1.0
2,6.410256,Some college,0,0.0
3,NaN,Less than HS,0,NaN
4,NaN,Some college,0,NaN
...,...,...,...,...
185482,NaN,HS diploma/GED,1,NaN
185483,23.557692,Graduate degree,0,1.0
185484,27.403847,College degree,1,1.0
185485,12.820513,Some college,0,1.0


In [29]:
cps.loc[cps['educ'] == 'HS diploma/GED', 'hrwage'].mean()


18.375734112709836

## Merging DataFrames

We will often find ourselves with variables spread across different datasets and files. We *merge* datasets together by matching up the two datasets on one or more variables. For example, I might have GDP data by country from the Penn World Tables, and demographic data by country from the World Bank. We would merge these two datasets and match up the observations by country. 

Some of the most powerful analysis comes from combining data from different sources. 

### Population Data

Let's load the 'city_pop.csv' file that contains some basic information about some major U.S. cities in 1990.

In [30]:
city_pop = pd.read_csv('data/city_pop.csv')
city_pop.head()

,city,population
0,New York,"8,008,278"
1,Los Angeles,"3,694,820"
2,Chicago,"2,896,016"
3,Houston,"1,953,631"
4,Philadelphia,"1,517,550"


Population data are already in good shape. 

### Merge the two DataFrames into one DataFrame
We want to match the two DataFrames together according to the city. 
In database-ese, we refer to variables we are matching as **keys.** In this case, the single key is city.

We also need to tell pandas how to treat keys that are not present in both databases. The different types of 'join' (more database-ese) are

1. **inner**: keep the intersection of the keys
2. **left**: keep all the keys from the left DataFrame
3. **right**: keep all the keys from right DataFrame
4. **outer**: keep all the keys from both DataFrames

We specify the join type with the `how` parameter. The default is inner, but for the sake of clarity, be explicit about your join.

Before we merge, let's review the two DataFrames.

In [31]:
city_pop.set_index(['city'], inplace=True)
city_pop.head()

,population
city,
New York,"8,008,278"
Los Angeles,"3,694,820"
Chicago,"2,896,016"
Houston,"1,953,631"
Philadelphia,"1,517,550"


We'll be merging on city, so let's re-index `mlb` so that city is our single index.

In [32]:
mlb.reset_index(['team','year'],inplace=True)
mlb.set_index('city', inplace=True)
mlb.head()

,team,year,win_prop
city,,,
Houston,Astros,1998,0.625767
Houston,Astros,1999,0.595092
Houston,Astros,2000,0.444444
Houston,Astros,2001,0.574074
Houston,Astros,2002,0.518519


In [33]:
print(mlb.head(2), '\n\n')
print(city_pop.head(2), '\n\n')
print('mlb has dim {0} and city_pop has dim {1}'.format(mlb.shape, city_pop.shape))

           team  year  win_prop
city                           
Houston  Astros  1998  0.625767
Houston  Astros  1999  0.595092 


            population
city                  
New York     8,008,278
Los Angeles  3,694,820 


mlb has dim (160, 3) and city_pop has dim (54, 1)


* Both DataFrames have an index named `city` that holds the city names.
* The city_pop DataFrame does not contain `year` since it contains information from only the 1990 Census.
* The DataFrames have different numbers of observations

Let's see how `.merge()` handles these issues. 

### Inner merge

* Use `left` and `right` to label the DataFrames to merge.
* Use `left_on` and `right_on` to identify the columns with keys.

In [35]:
mlb_inner = pd.merge(left=mlb, right=city_pop, left_on=['city'], right_on=['city'], how='inner')
mlb_inner.head()

,team,year,win_prop,population
city,,,,
Houston,Astros,1998,0.625767,"1,953,631"
Houston,Astros,1999,0.595092,"1,953,631"
Houston,Astros,2000,0.444444,"1,953,631"
Houston,Astros,2001,0.574074,"1,953,631"
Houston,Astros,2002,0.518519,"1,953,631"


A few things to note.

- The population of each city gets repeated for each year, because it comes from only the 1990 Census.
- The merged DataFrame has 150 rows&mdash;fewer than `mlb` started with. There is one city in `mlb` that is not included in city_pop. Those rows are not included in the merged DataFrame when we use an inner merge. 

### Outer merge

An outer merge keeps all the keys&mdash;we do not lose any observations. I am adding the `indicator` to help us see how the two DataFrames compare. 

In [36]:
mlb_outer = pd.merge(left=mlb, right=city_pop, 
                      left_on=['city'], right_on=['city'], 
                      how='outer', indicator=True)
mlb_outer.head()

,team,year,win_prop,population,_merge
city,,,,,
Albuquerque,NaN,NaN,NaN,"448,607",right_only
Arlington,NaN,NaN,NaN,"332,969",right_only
Atlanta,Braves,1998.0,0.650307,"416,474",both
Atlanta,Braves,1999.0,0.631902,"416,474",both
Atlanta,Braves,2000.0,0.586420,"416,474",both


Let's check the `indicator` variable. In particular, we want to see which rows were present in city_pop only, and which were present in mlb only.

In [37]:
mlb_outer_ro=mlb_outer[mlb_outer['_merge']=='right_only']
mlb_outer_ro.head()

,team,year,win_prop,population,_merge
city,,,,,
Albuquerque,NaN,NaN,NaN,"448,607",right_only
Arlington,NaN,NaN,NaN,"332,969",right_only
Austin,NaN,NaN,NaN,"656,562",right_only
Baltimore,NaN,NaN,NaN,"651,154",right_only
Boston,NaN,NaN,NaN,"589,141",right_only


In [38]:
mlb_outer_lo=mlb_outer[mlb_outer['_merge']=='left_only']
mlb_outer_lo.head()

,team,year,win_prop,population,_merge
city,,,,,
Cincinnati,Reds,1998.0,0.472393,NaN,left_only
Cincinnati,Reds,1999.0,0.588957,NaN,left_only
Cincinnati,Reds,2000.0,0.524691,NaN,left_only
Cincinnati,Reds,2001.0,0.407407,NaN,left_only
Cincinnati,Reds,2002.0,0.481481,NaN,left_only


The `_merge` variable helps us understand what happened with the inner merge. Cincinnati is only in `mlb`, while many cities like Albuquerque and Arlington are only in `city_pop`.

Notice that pandas filled in NaN where needed. Nice.

### Left merge

A left merge only keeps keys from the left data set. Let's see what happens.

In [40]:
mlb_left = pd.merge(left=mlb, right=city_pop, 
                      left_on=['city'], right_on=['city'], 
                      how='left', indicator=True)
mlb_left.head()

,team,year,win_prop,population,_merge
city,,,,,
Houston,Astros,1998,0.625767,"1,953,631",both
Houston,Astros,1999,0.595092,"1,953,631",both
Houston,Astros,2000,0.444444,"1,953,631",both
Houston,Astros,2001,0.574074,"1,953,631",both
Houston,Astros,2002,0.518519,"1,953,631",both


In [41]:
mlb_left_lo = mlb_left[mlb_left['_merge']=='left_only']
mlb_left_lo.head()

,team,year,win_prop,population,_merge
city,,,,,
Cincinnati,Reds,1998,0.472393,NaN,left_only
Cincinnati,Reds,1999,0.588957,NaN,left_only
Cincinnati,Reds,2000,0.524691,NaN,left_only
Cincinnati,Reds,2001,0.407407,NaN,left_only
Cincinnati,Reds,2002,0.481481,NaN,left_only


The left merge keeps the Cincinnati Reds even though it doesn't have a match in `city_pop`.

In [42]:
mlb_left_ro = mlb_left[mlb_left['_merge']=='right_only']
mlb_left_ro.head()

,team,year,win_prop,population,_merge
city,,,,,


By construction, the left merge doesn't keep any 'right_only' observations!

### Right merge

This is the reverse of a left merge (in some sense, they are the same, since we decide which dataset is "left" or "right"). We'll skip for now because it doesn't make much sense in this context.

### Merging on different units of observation

The above example is what we call a *many-to-one* merge: the keys in the left dataset (mlb) are not unique, but the keys in the right dataset (city_pop) are.

Merging data can get complicated! In pandas, we can use the `validate` option make sure we understand the units of observation in each DataFrame. (If you chose an option that is incompatible with the attempted merge, it spits an error.)

In [43]:
mlb_val = pd.merge(left=mlb, right=city_pop, 
                      left_on=['city'], right_on=['city'], 
                      how='outer', indicator=True, validate='many_to_one')
mlb_val.head()

,team,year,win_prop,population,_merge
city,,,,,
Albuquerque,NaN,NaN,NaN,"448,607",right_only
Arlington,NaN,NaN,NaN,"332,969",right_only
Atlanta,Braves,1998.0,0.650307,"416,474",both
Atlanta,Braves,1999.0,0.631902,"416,474",both
Atlanta,Braves,2000.0,0.586420,"416,474",both


## More practice: merging

In this practice, we will merge several DataFrames into one.

- 'month_legal.csv' : Month in which sports betting became legal in each state
- 'delinquency.csv': Monthly home mortgage delinquency rates by state
- 'state_unemp.csv': Monthly unemployment rates by state

1. Use `.read_csv()` to read each file as a DataFrame.
   
2. Examine the three datasets:
    - What is the level of observation?
    - Are the variable types consistent across all three datasets?

In [1]:
import pandas as pd

month_legal = pd.read_csv("data/month_legal.csv")
delinquency = pd.read_csv("data/delinquency.csv")
state_unemp = pd.read_csv("data/state_unemp.csv")

month_legal
delinquency
state_unemp


FileNotFoundError: [Errno 2] No such file or directory: 'data/month_legal.csv'

    3. Merge 'delinquency.csv' and 'state_unemp.csv' into one DataFrame. Does it matter whether you use inner, outer, left, or right? Why or why not?

In [57]:
state_unemp.head(5)

,DATE,State,UnempRate
0,2010-01-01,Alabama,11.1
1,2010-01-01,Alaska,8.4
2,2010-01-01,Arizona,10.6
3,2010-01-01,Arkansas,7.9
4,2010-01-01,California,12.6


4. Merge 'month_legal.csv' with the newly merged data set. Does it matter whether you use inner, outer, left, or right? Why or why not?

5. Create an indicator called 'legal' that =0 if betting is not legal in the current state and month, and =1 if betting is legal in each state-month observation.

\[Hint: Create a new column by comparing 'date' and 'MonthLegal' using `>=` , and then cast `.astype(int)`. 'date' and 'MonthLegal' are both dates and this comparison will work as though we explicitly made them datetime objects. We will learn more about datetime objects soon!\]